In [3]:
! pip install opencv-python

     --------------------------------------- 35.6/35.6 MB 19.8 MB/s eta 0:00:00


In [19]:
os.getcwd()

'C:\\Users\\admin'

In [22]:
import os
import re
import cv2
import time
import numpy as np
import pandas as pd
import json
from sklearn.model_selection import train_test_split
from PIL import Image, ImageDraw

from retinanet import model
from retinanet.dataloader import collater, Resizer, Augmenter, Normalizer, UnNormalizer, CSVDataset
import torch
import torch.optim as optim
import torchvision.transforms as T
from torchvision.utils import make_grid 
from torch.utils.data import DataLoader, Dataset


import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

ModuleNotFoundError: No module named 'retinanet'

In [7]:
# Get data from annotations
path_to_anno = 'C:/dev/MLDL/미니프로젝트/data/annotations.json'

with open(path_to_anno) as f:
  data = json.load(f)

print(type(data))
print()

# required classes for my problem statement
classes_req = ['Bottle', 'Can', 'Cup', 'Paper']
classes = {}
classes_id = {}

for i in data['categories']:
  if(i['supercategory'] in classes_req):
    classes[i['id']] = i['name']

print(classes)
print("Number of classes: ", len(classes))
print()

#Get image path and data 
images_data = {}

for i in data['images']:
  temp = {
      'file_n': 'C:/dev/MLDL/미니프로젝트/data/'+i['file_name'],
      'width': i['width'],
      'height': i['height'],
  }
  images_data[i['id']] = temp
print("Image data gathered")
print()

#Check if bounding box is valid
def is_valid(x_min, y_min, x_max, y_max, width, height):
    return (x_min in range(0, width)) and (x_max in range(0, width)) and (y_min in range(0, height)) and (y_max in range(0, width))


filter_class = {
    'Other plastic bottle': "Bottle", 
    'Clear plastic bottle': "Bottle",  
    'Glass bottle': 'Bottle',
    'Food Can': 'Can', 
    'Aerosol': 'Can', 
    'Drink can': 'Can', 
    'Paper cup': 'Cup', 
    'Disposable plastic cup': 'Cup', 
    'Foam cup': 'Cup', 
    'Glass cup': 'Cup', 
    'Other plastic cup': 'Cup', 
    'Magazine paper': 'Paper', 
    'Tissues': 'Paper', 
    'Wrapping paper': 'Paper', 
    'Normal paper': 'Paper'}

    
# Group images with their data
images_with_anno = []
not_valid=0
for i in data['annotations']:
  if(i['category_id'] in classes.keys()):
    img_d = images_data[i['image_id']]
    x_min= int(i['bbox'][0])
    y_min= int(i['bbox'][1])
    x_max= int(i['bbox'][0]+i['bbox'][2])
    y_max= int(i['bbox'][1]+i['bbox'][3])
    valid = is_valid(x_min, y_min, x_max, y_max, img_d['width'], img_d['height'])
    temp = {
        'image_name': img_d['file_n'],
        'x_min': int(i['bbox'][0]),
        'y_min': int(i['bbox'][1]),
        'x_max': int(i['bbox'][0]+i['bbox'][2]),
        'y_max': int(i['bbox'][1]+i['bbox'][3]),
        'class_name': filter_class[classes[i['category_id']]],
    }
    if(valid):
        images_with_anno.append(temp)
    else:
        not_valid+=1

print("Complete dict generated", "\nFound not valid: ", not_valid)
print()

<class 'dict'>

{4: 'Other plastic bottle', 5: 'Clear plastic bottle', 6: 'Glass bottle', 10: 'Food Can', 11: 'Aerosol', 12: 'Drink can', 20: 'Paper cup', 21: 'Disposable plastic cup', 22: 'Foam cup', 23: 'Glass cup', 24: 'Other plastic cup', 30: 'Magazine paper', 31: 'Tissues', 32: 'Wrapping paper', 33: 'Normal paper'}
Number of classes:  15

Image data gathered

Complete dict generated 
Found not valid:  184



In [8]:
df = pd.DataFrame(images_with_anno)
train_df, test_df = train_test_split(df, test_size=0.2)
train_df.to_csv('annotations.csv', index=False, header=None)
test_df.to_csv('test_annotations.csv', index=False, header=None)

In [9]:
with open('classes.csv', 'w') as f:
  for i, (class_name) in enumerate(train_df['class_name'].unique()):
    f.write(f'{class_name},{i}\n')

In [10]:
train_data = CSVDataset('./annotations.csv', './classes.csv', transform = T.Compose([Augmenter(), Normalizer(), Resizer()]))
test_data = CSVDataset('./test_annotations.csv', './classes.csv', transform = T.Compose([Augmenter(), Normalizer(), Resizer()]))

NameError: name 'CSVDataset' is not defined